In [ ]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
model = "meta-llama/Meta-Llama-3-8B-Instruct"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

### Acknowledgement

Structuring bibliographical entries task taken from the excellent [marginalia project](https://github.com/Pleias/marginalia/tree/main)'s example notebooks.

In [ ]:
def format_history(message, history, system_prompt):
    chat_history = [{"role": "system", "content": system_prompt}]
    for query, response in history:
        chat_history.append({"role": "user", "content": query})
        chat_history.append({"role": "assistant", "content": response})
    chat_history.append({"role": "user", "content": message})
    return chat_history


def prompt_model(message, history=None, system_prompt="You are a helpful assistant."):
    chat_history = format_history(message, history if history else [], system_prompt)
    response = client.chat.completions.create(model=model, messages=chat_history)
    return response.choices[0].message.content

## Zero-shot prompting

In [ ]:
prompt = """
Transform this a book entry into structured bibliographic data

Extract the following bibliographic fields:
- the author(s) of the book, which can be expressed with a possessive like Hobbes's ("author")
- the title of the book ("title")
- the translator(s) of the book ("translator")
- the date of publication ("date")
- the place of publication ("place")
- any information related to the format such as volumes, folios ("format") 
- any other information related to the book ("other")

Return the results as a valid JSON object structured like this : {"author": "…", "title": "…", "translator": "…", "date": "…", "place": "…", "format": "…", "other": "…"}

The book entry:
```
FINE large Folio BIBLE, compleat, Oxford 1727.
```
"""

print(
    prompt_model(
        prompt, system_prompt="You are a powerful annotator of bibliographic data"
    )
)

## Few-shot prompting

In [ ]:
good_answer = """
Here is the extracted bibliographic data in JSON format:

```
{
  "author": null,
  "title": "BIBLE",
  "translator": null,
  "date": "1727",
  "place": "Oxford",
  "format": "Folio",
  "other": "compleat"
}
```
"""

rebellion_question = """
Another book entry:
```
Clarendon's History of the Rebellion, 3 Vol
```
"""

system_prompt = "You are a powerful annotator of bibliographic data"

history = [
    (intial_prompt, good_answer),
]

print(prompt_model(rebellion_question, history, system_prompt))

**Exercise** Use "History of the Rebellion" as a few-shot example as well by defining the `rebellion_answer` variable in the cell below. Then try to analyze book entry "Wiquefort's compleat Ambassador, translated by Digby, finely bound." by defining `ambassador_question`.

In [ ]:
rebellion_answer = """
<your answer here>
"""

ambassador_question = """
<your question regarding 'Wiquefort's compleat Ambassador, translated by Digby, finely bound.' here>
"""

history = [(prompt, good_answer), (rebellion_question, rebellion_answer)]

prompt_model(ambassador_question, history, system_prompt)

**Exercise** Add a final example by defining `ambassador_answer` in the cell below. Test your prompt on the entry "Hobbes's Leviathan, very scarce." by defining `new_question`.

In [ ]:
ambassador_answer = """
<your answer regarding "Wiquefort's compleat Ambassador, translated by Digby, finely bound." here>
"""

new_question = """
<your question regarding "Hobbes's Leviathan, very scarce." here>
"""

history = [
    (prompt, good_answer),
    (rebellion_question, rebellion_answer),
    (ambassador_question, ambassador_answer),
]

prompt_model(new_question, history, system_prompt)